In [6]:
import os
import json
import torch
import numpy as np
import xml.etree.ElementTree as ET
from pathlib import Path
from PIL import Image

def convert_voc_to_custom(xml_folder, image_folder, output_json):
    annotations = []

    for xml_file in os.listdir(xml_folder):
        if xml_file.endswith('.xml'):
            tree = ET.parse(os.path.join(xml_folder, xml_file))
            root = tree.getroot()
            filename = root.find('filename').text
            image_path = os.path.join(image_folder, filename)

            for obj in root.findall('object'):
                category = obj.find('name').text
                bndbox = obj.find('bndbox')
                x1 = int(bndbox.find('xmin').text)
                y1 = int(bndbox.find('ymin').text)
                x2 = int(bndbox.find('xmax').text)
                y2 = int(bndbox.find('ymax').text)

                annotations.append({
                    "file_name": filename,
                    "category": category,
                    "bbox": [x1, y1, x2, y2]
                })

    with open(output_json, 'w') as f:
        json.dump(annotations, f, indent=4)

def calculate_metrics(detections, annotations, iou_threshold=0.5):
    TP = 0
    FP = 0
    FN = 0
    iou_list = []

    for detection in detections:
        detected_bbox = detection['bbox']
        detected_filename = detection['file_name']
        matched = False
        for annotation in annotations:
            if annotation['file_name'] == detected_filename:
                true_bbox = annotation['bbox']
                iou = compute_iou(detected_bbox, true_bbox)
                if iou >= iou_threshold:
                    TP += 1
                    matched = True
                    iou_list.append(iou)
                    break
        if not matched:
            FP += 1
    
    FN = len(annotations) - TP

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    map_score = np.mean(iou_list) if iou_list else 0

    return precision, recall, f1_score, map_score

def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)

    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    iou = interArea / float(boxAArea + boxBArea - interArea)

    return iou

xml_folder = 'D:/test3/container_dataset/test_xml'
image_folder = 'D:/test3/container_dataset/test/images'
output_json = 'D:/test3/yolov5_test/annotations_custom.json'
model_path = "D:/test3/exp4/weights/best.pt"
test_image_folder = 'D:/test3/container_dataset/test/images'
detections_json = 'D:/test3/yolov5_test/detections_custom.json'

convert_voc_to_custom(xml_folder, image_folder, output_json)
print("XML 轉換成功完成")

model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

test_images = [f for f in os.listdir(test_image_folder) if f.endswith(".jpg") or f.endswith(".png")]

all_detections = []

for image_file in test_images:
    image_path = os.path.join(test_image_folder, image_file)
    results = model(image_path)
    predictions = results.xyxy[0].cpu().numpy()

    for det in predictions:
        if det[4] > 0.5:
            x1, y1, x2, y2, score, class_id = det
            all_detections.append({
                "file_name": image_file,
                "category": model.names[int(class_id)],
                "bbox": [float(x1), float(y1), float(x2), float(y2)],
                "score": float(score)
            })

print("Detections:", all_detections)

with open(detections_json, 'w') as f:
    json.dump(all_detections, f, indent=4)

with open(output_json, 'r') as f:
    annotations = json.load(f)

precision, recall, f1_score, map_score = calculate_metrics(all_detections, annotations)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1_score:.4f}")
print(f"mAP: {map_score:.4f}")

XML 转换为自定义格式完成


Using cache found in C:\Users\brain/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-5-25 Python-3.11.1 torch-2.1.0.dev20230307+cu118 CUDA:0 (NVIDIA GeForce RTX 2060 with Max-Q Design, 6144MiB)

Fusing layers... 


WARNING ⚠️ torchvision==0.15 is incompatible with torch==2.1.
Run 'pip install torchvision==0.16' to fix torchvision or 'pip install -U torch torchvision' to update both.
For a full compatibility table see https://github.com/pytorch/vision#installation


Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


Detections: [{'file_name': 'image_0001.jpg', 'category': 'container', 'bbox': [746.1029052734375, 110.5165786743164, 1111.27685546875, 236.30606079101562], 'score': 0.9040830731391907}, {'file_name': 'image_0002.jpg', 'category': 'container', 'bbox': [742.2112426757812, 153.30421447753906, 1164.455322265625, 296.79010009765625], 'score': 0.8691405057907104}, {'file_name': 'image_0003.jpg', 'category': 'container', 'bbox': [749.3065795898438, 151.9781494140625, 1163.673095703125, 291.0323791503906], 'score': 0.8750637769699097}, {'file_name': 'image_0004.jpg', 'category': 'container', 'bbox': [782.1802978515625, 94.86641693115234, 1128.1429443359375, 212.23336791992188], 'score': 0.9070493578910828}, {'file_name': 'image_0005.jpg', 'category': 'container', 'bbox': [824.5478515625, 41.8655891418457, 1096.3065185546875, 137.64306640625], 'score': 0.9199814200401306}, {'file_name': 'image_0006.jpg', 'category': 'container', 'bbox': [851.65478515625, 4.8189697265625, 1072.2994384765625, 84.